On a 4000 kg de fontes dans le four de Coulée avec un pourcentage de Mg = 0.0045 %.
On consonmme 36 Kg de fonte par minute dans le four de Coulée.
On perd 0.005 % de Mg par minute.

Tu es un mathématicien et tu dois Tracer la courbe d'évolution du Mg
dans le four de Coulée. 

Si tu as des questions ou s'il te manque certaines informations, n'hésite pas à me les demander !!

In [1]:
# Liste des cadences de production par heure pour chaque modèle
liste_cadences_moule_par_heure = [190, 190, 190, 210, 190, 190, 190, 190, 190]  # en unités/heure
# Liste des quantités de moules à produire pour chaque modèle
liste_quantites_moules_a_produire = [121, 50, 35, 10, 100, 50, 35, 50, 40]  # en unités
# Liste des masses de grappes de moules pour chaque modèle
liste_masses_grappes_moules = [41.2, 39.36, 15.6, 19.94, 41.20, 39.36, 15.60, 41.20, 29.65]  # en kg



# Cadence de production du i-ème modèle : le nombre de moules par heure
cadence_moule_par_heure = 190  # en unités/heure

# Masse des grappes de moules du i-ème modèle
masse_grappe_moule = 41.2  # en kg

# Quantité de moules du i-ème modèle à produire
quantite_moules_a_produire = 121  # en unités

# La cadence de consommation en kg/min du i-ème modèle
cadence_consommation_kg_min = cadence_moule_par_heure / 60 * masse_grappe_moule  # en kg/min

# Quantité de moules du i-ème modèle produits en une minute
cadence_moule_par_minute = cadence_consommation_kg_min / masse_grappe_moule  # en unités/min

# Quantité de moules restants du i-ème modèle après une minute
quantite_moules_restants = quantite_moules_a_produire - cadence_moule_par_minute # en unités


In [2]:

# Temps nécessaire pour produire tous les moules du i-ème modèle
temps_total_minutes = quantite_moules_a_produire / cadence_moule_par_minute

temps_total_minutes, quantite_moules_a_produire - 38*cadence_moule_par_minute

(38.21052631578947, 0.6666666666666714)

In [10]:
Soit PFC_Consom, la fonte utiliser pour produire les moules 
Soit Mg, le pourcentage de Mg une  variable qui intervient lors de la production des moules
Que l'on conservera dans un dictionnaire : times_dict, PFC_Consomvalues_dict, Mgvalues_dict

Soit temps_serie, le temps mis pour passer au modeles suivant

Soit PFCmin et Mgmin, la masse de fonte minimal et le pourcentage de Mg minimal
que peuvent prendre PFC_Consom et Mg


A chaque temps t, on a 4 états possibles:
 - Soit on produit des moules 
 - Soit on change de modeles
 - Soit on est en Panne
 - Soit on est à la Fin

Lors que l'on produit des moules :

    Si etat est egal à etat_Consom alors # On verifie que l'on est en etat consommation dans le temps

        si PFC_Consom - cadence_consommation_kg_min et Mg - 0.0005  <= PFCmin et Mgmin alors  # Si l'on peut consommer alors on le fait !!
            
            PFC_Consom -= cadence_consommation_kg_min et Mg -= 0.0005

            # Quantité de moules du i-ème modèle produits en une minute
            cadence_moule_par_minute = cadence_consommation_kg_min / masse_grappe_moule  # en unités/min

            # Quantité de moules restants du i-ème modèle après une minute
            quantite_moules_restants = quantite_moules_a_produire - cadence_moule_par_minute # en unités

            # on met à jour liste_quantites_moules_a_produire et liste_masses_grappes_moules
            si quantite_moules_restants est nul alors 
                - Le temps passe a l'etat changement de Modeles
                - on passe au modele suivant dans les listes : 
                    # Suppression des éléments déjà traités dans les listes
                    liste_masses_grappes_moules.pop(0)
                    liste_quantites_moules_a_produire.pop(0)
                    liste_cadences_moule_par_heure.pop(0)
            sinon 
                - remplacer quantite_moules_a_produire par quantite_moules_restants
                - Le temps reste a l'etat on produit des moules

            si liste_masses_grappes_moules est vide alors 
                - Le temps passe a l'etat Fin ( on arrete toute la proccedure )
                

        Sinon # Si l'on ne peut pas consommer alors on rajoute de la fonte venant de la poche et on consomme
            PFC_Consom += 1250 et Mg = 0.0045 # on rajoute de la fonte venant de la poche 

            # Maintenant on peut consommer !!
            PFC_Consom -= cadence_consommation_kg_min et Mg -= 0.0005

            # Quantité de moules du i-ème modèle produits en une minute
            cadence_moule_par_minute = cadence_consommation_kg_min / masse_grappe_moule  # en unités/min

            # Quantité de moules restants du i-ème modèle après une minute
            quantite_moules_restants = quantite_moules_a_produire - cadence_moule_par_minute # en unités

            # on met à jour liste_quantites_moules_a_produire et liste_masses_grappes_moules
            si quantite_moules_restants est nul alors 
                - Le temps passe a l'etat changement de Modeles
                - on passe au modele suivant dans les listes : 
                    # Suppression des éléments déjà traités dans les listes
                    liste_masses_grappes_moules.pop(0)
                    liste_quantites_moules_a_produire.pop(0)
                    liste_cadences_moule_par_heure.pop(0)
            sinon 
                - remplacer quantite_moules_a_produire par quantite_moules_restants
                - Le temps reste a l'etat on produit des moules

            si liste_masses_grappes_moules est vide alors 
                - Le temps passe a l'etat Fin ( on arrete toute la proccedure )



Lors que l'on change de modeles :

    Si etat est egal à etat_Serie alors # On verifie que l'on est en etat Changement de Serie dans le temps
        Mg -= 0.0005 # Mg perd son pourcentage
        temps_serie -= 1 # On decremente temps_serie de 1   

        Si Mg  <= Mgmin : # Si le Mg n'est plus bon on doit ajouter de la fonte
            PFC_Consom += 1250 et Mg = 0.0045 # on rajoute de la fonte venant de la poche 
    
        Si PFC_Consom >= PFCMax :
            "Error on est bloqué"

        Si temps_serie est nul alors 
            - Le temps passe à l'état produire des moules
        Sinon 
            - Le temps reste à temps_serie 



Lors que l'on est en Panne :

    Si etat est egal à etat_Panne alors # On verifie que l'on est en etat panne dans le temps
        Mg -= 0.0005 #Mg perd son pourcentage
        temps_Panne -= 1 # On decremente temps_Panne de 1   
        Si temps_Panne est nul alors 
            - Le temps passe à l'état produire des moules
        Sinon 
            - Le temps reste à temps_panne 



Remarques : 
    L'utilisateur doit avoir un bouton pour pouvoir activer l'etat en Panne


SyntaxError: unterminated string literal (detected at line 3) (1128451648.py, line 3)

In [21]:
# Données initiales
liste_cadences_moule_par_heure = [190, 190, 190, 210, 190, 190, 190, 190, 190]  # en unités/heure
liste_quantites_moules_a_produire = [121, 50, 35, 10, 100, 50, 35, 50, 40]  # en unités
liste_masses_grappes_moules = [41.2, 39.36, 15.6, 19.94, 41.20, 39.36, 15.60, 41.20, 29.65]  # en kg

# Variables de simulation
PFCmin = 2500 # en kg
PFCmax = 5000 # en kg
Mgmin = 0.035 # en %
Mgmax = 0.045 # en %
eC = 0.0005 # en %
poche_i = 1250 # en kg
tmin = 0 # en minutes
t = 0 # en minutes


temps_serie = 10  # Initialisation du temps pour changer de modèle
temps_Panne = 10  # Initialisation du temps pour la panne

# Dictionnaires pour stocker les variables et états
cles = ["chargement_et_fusion", "Four 1", "Four 2", "Four 3", "Four 4", "Four 5"]
times_dict = {cle: [] for cle in cles}
PFC_Consomvalues_dict = {cle: [] for cle in cles}
Mgvalues_dict = {cle: [] for cle in cles}


PFC_Consom = 3750  # Initialisation de la fonte consommée
Mg =  Mgmax  # Initialisation du pourcentage de Mg
tmax = 4*60 # en minutes

# Initialisation des états
etat = "etat_Consom"  # État initial : production de moules

temps_serie_courant = temps_serie
temps_Panne_courant = temps_Panne
numPoche = 3
numFours = 1



#  if  debut < t and  t > fin :
debut,fin = 73,73


def Consommation(Mg, PFC_Consom, quantite_moules_a_produire, cadence_consommation_kg_min, cadence_moule_par_minute):
    
    # Mise à jour du pourcentage de Mg et du poids fonte coulée
    PFC_Consom -= cadence_consommation_kg_min
    Mg -= eC

    if PFC_Consom > PFCmax:
        print("On a dépassé la capacité du Four de Coulée")
    

    quantite_moules_restants = quantite_moules_a_produire - cadence_moule_par_minute

    if debut < t < fin:
        print("Ici2", Mg, PFC_Consom, cadence_consommation_kg_min, quantite_moules_a_produire, quantite_moules_restants)

    # On a fini de réaliser le i-ème modèle alors on passe au i+1-ème modèle
    if quantite_moules_restants <= 0:
        etat = "etat_Serie"
        # Suppression des éléments déjà traités dans les listes
        liste_masses_grappes_moules.pop(0)
        liste_quantites_moules_a_produire.pop(0)
        liste_cadences_moule_par_heure.pop(0)

        # Si on a fini de tout produire alors on stoppe la procédure 
        # On passe à l'état Fin
        if not liste_masses_grappes_moules:
            etat = "etat_Fin"

        if debut < t < fin:
            print("Ici3", Mg, PFC_Consom, cadence_consommation_kg_min, quantite_moules_a_produire, quantite_moules_restants)

    # On a fini de réaliser le j-ème moule du i-ème modèle alors on passe au j-ème moule
    # En restant dans l'état consommation 
    else:
        if debut < t < fin:
            print("Ici3", Mg, PFC_Consom, cadence_consommation_kg_min, quantite_moules_a_produire, quantite_moules_restants)

        liste_quantites_moules_a_produire[0] = quantite_moules_restants
        etat = "etat_Consom"
    
    return Mg, PFC_Consom, etat


# Boucle de simulation
while t < tmax :
    if etat == "etat_Consom":
        masse_grappe_moule = liste_masses_grappes_moules[0] 
        cadence_moule_par_heure = liste_cadences_moule_par_heure[0]
        quantite_moules_a_produire  = liste_quantites_moules_a_produire[0]
        
        # La cadence de consommation en kg/min du i-ème modèle
        cadence_consommation_kg_min = cadence_moule_par_heure / 60 * masse_grappe_moule  # en kg/min

        # Quantités de moules du i-ème modèle produits en une minute
        cadence_moule_par_minute = cadence_consommation_kg_min / masse_grappe_moule # en unités/min
        
        if  debut < t and  t < fin :
            print('t = ',t)
            print("Ici1", Mg, PFC_Consom, cadence_consommation_kg_min,quantite_moules_a_produire,quantite_moules_restants)
        
        # Si l'on peut consommer (c-a-d Mg et PFC sont okay) alors on consomme
        if (PFC_Consom - cadence_consommation_kg_min >= PFCmin) and (Mg - eC >= Mgmin):
            
            Mg, PFC_Consom, etat = Consommation(Mg, PFC_Consom, quantite_moules_a_produire, cadence_consommation_kg_min, cadence_moule_par_minute)
                
        else:
            # Ajout poche car Mg ou PFC insuffisant !!
            PFC_Consom += poche_i
            Mg = Mgmax 

            if numPoche < 4:
                numPoche += 1
            else :
                numPoche = 1
                numFours += 1
                cle = f"Four {numFours-1}" 
                PFC_Consomvalues_dict[cle].append(PFC_Consom)
                Mgvalues_dict[cle].append(Mg)
                times_dict[cle].append(t)



            if PFC_Consom > PFCmax :
                print("On a depasser pas la capacité du Fours de Coulée")
            
            Mg, PFC_Consom, etat = Consommation(Mg, PFC_Consom, quantite_moules_a_produire, cadence_consommation_kg_min, cadence_moule_par_minute)
            

    elif etat == "etat_Serie":
        # print("Changement de Serie")
        Mg -= eC
        temps_serie_courant -= 1
        
        if temps_serie_courant <= 0:
            etat = "etat_Consom"
            if  debut < t and  t < fin :
                print("Fin Temps Serie",Mg, PFC_Consom, cadence_consommation_kg_min,quantite_moules_restants)
            temps_serie_courant = temps_serie
        else:
            etat == "etat_Serie" 


    elif etat == "etat_Panne":
        Mg -= eC
        temps_Panne_courant -= 1
        
        if temps_Panne_courant <= 0:
            etat = "etat_Consom"
            if  debut < t and  t < fin :
                print("Fin Panne",Mg, PFC_Consom, cadence_consommation_kg_min,quantite_moules_restants)
            temps_Panne_courant = temps_Panne
        else:
            etat == "etat_Panne"

    elif etat == "etat_Fin":
        break
    
    # Mise à jour du temps
    t += 1
    cle = "Four 1"
    cle = f"Four {numFours}" 
    PFC_Consomvalues_dict[cle].append(PFC_Consom)
    Mgvalues_dict[cle].append(Mg)
    times_dict[cle].append(t)




# Affichage des résultats finaux ou mise à jour des dictionnaires
print("Résultats finaux ou mises à jour des dictionnaires : ")
print("times_dict : ", times_dict)
print("PFC_Consomvalues_dict : ", PFC_Consomvalues_dict)
print("Mgvalues_dict : ", Mgvalues_dict)

On a depasser pas la capacité du Fours de Coulée
Résultats finaux ou mises à jour des dictionnaires : 
times_dict :  {'chargement_et_fusion': [], 'Four 1': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 19], 'Four 2': [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 76], 'Four 3': [77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 154], 'Four 4': [155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [22]:
import plotly.graph_objects as go

def Plot_consom(PFC_values_dict, Mgvalues_dict, times_dict, PFCmin, PFCmax, Mgmin, Mgmax):
    # Dictionnaire de couleurs pour chaque clé
    color_dict = {
        "chargement_et_fusion": 'royalblue',
        "Four 1": 'firebrick',
        "Four 2": 'green',
        "Four 3": 'orange',
        "Four 4": 'purple',
        "Four 5": 'royalblue'
    }

    # Create Plotly figure for PFC
    fig_pfc = go.Figure()

    # Add PFC lines for each key
    for key in PFC_values_dict.keys():
        fig_pfc.add_trace(go.Scatter(x=times_dict[key], y=PFC_values_dict[key], mode='lines', name=f'PFC over time ({key})', line=dict(color=color_dict[key])))

    # Add horizontal lines for PFCmin and PFCmax
    tmin, tmax = times_dict["Four 1"][0],times_dict["Four 1"][-1]
    fig_pfc.add_shape(type="line", x0=tmin, y0=PFCmin, x1=tmax, y1=PFCmin, line=dict(color="firebrick", width=2, dash="dash"), name="PFCmin")
    fig_pfc.add_shape(type="line", x0=tmin, y0=PFCmax, x1=tmax, y1=PFCmax, line=dict(color="green", width=2, dash="dash"), name="PFCmax")

    # Customize layout for PFC figure
    fig_pfc.update_layout(title='Evolution de la fonte dans le Four de Coulée au cours du temps (PFC)', 
                          xaxis_title='Time (minutes)', yaxis_title='PFC (kg)', 
                          template='plotly', showlegend=True,
                          annotations=[
                              dict(x=tmax, y=PFCmin, xref="x", yref="y", text="PFCmin", showarrow=True, arrowhead=2, ax=20, ay=-30),
                              dict(x=tmax, y=PFCmax, xref="x", yref="y", text="PFCmax", showarrow=True, arrowhead=2, ax=20, ay=-30)
                          ])

    # Show the PFC plot
    fig_pfc.show()

    # Create Plotly figure for Mg
    fig_mg = go.Figure()

    # Add Mg lines for each key
    for key in Mgvalues_dict.keys():
        fig_mg.add_trace(go.Scatter(x=times_dict[key], y=Mgvalues_dict[key], mode='lines', name=f'Mg over time ({key})', line=dict(color=color_dict[key])))

    # Add horizontal lines for Mgmin and Mgmax
    fig_mg.add_shape(type="line", x0=tmin, y0=Mgmin, x1=tmax, y1=Mgmin, line=dict(color="firebrick", width=2, dash="dash"), name="Mgmin")
    fig_mg.add_shape(type="line", x0=tmin, y0=Mgmax, x1=tmax, y1=Mgmax, line=dict(color="green", width=2, dash="dash"), name="Mgmax")

    # Customize layout for Mg figure
    fig_mg.update_layout(title='Evolution de la fonte dans le Four de Coulée au cours du temps (Mg)', 
                         xaxis_title='Time (minutes)', yaxis_title='Mg (kg)', 
                         template='plotly', showlegend=True,
                         annotations=[
                              dict(x=tmax, y=Mgmin, xref="x", yref="y", text="Mgmin", showarrow=True, arrowhead=2, ax=20, ay=-30),
                              dict(x=tmax, y=Mgmax, xref="x", yref="y", text="Mgmax", showarrow=True, arrowhead=2, ax=20, ay=-30)
                          ])

    # Show the Mg plot
    fig_mg.show()

# Call the function with the appropriate dictionaries and parameters
Plot_consom(PFC_Consomvalues_dict, Mgvalues_dict, times_dict, PFCmin, PFCmax, Mgmin, Mgmax)


In [12]:
# Données initiales
liste_cadences_moule_par_heure = [190, 190, 190, 210, 190, 190, 190, 190, 190]  # en unités/heure
liste_quantites_moules_a_produire = [121, 50, 35, 10, 100, 50, 35, 50, 40]  # en unités
liste_masses_grappes_moules = [41.2, 39.36, 15.6, 19.94, 41.20, 39.36, 15.60, 41.20, 29.65]  # en kg

# Variables de simulation
PFCmin = 2500 # en kg
PFCmax = 5000 # en kg
Mgmin = 0.035 # en %
Mgmax = 0.045 # en %
eC = 0.0005 # en %
poche_i = 1250 # en kg
tmin = 0 # en minutes
t = 0 # en minutes


temps_serie = 10  # Initialisation du temps pour changer de modèle
temps_Panne = 10  # Initialisation du temps pour la panne

# Dictionnaires pour stocker les variables et états
cles = ["chargement_et_fusion", "Four 1", "Four 2", "Four 3", "Four 4", "Four 5"]
times_dict = {cle: [] for cle in cles}
PFC_Consomvalues_dict = {cle: [] for cle in cles}
Mgvalues_dict = {cle: [] for cle in cles}


PFC_Consom = 3750  # Initialisation de la fonte consommée
Mg =  Mgmax  # Initialisation du pourcentage de Mg
tmax = 4*60 # en minutes

# Initialisation des états
etat = "etat_Consom"  # État initial : production de moules
temps_serie_courant = temps_serie
temps_Panne_courant = temps_Panne
numPoche = 3
numFours = 1


#  if  debut < t and  t > fin :
debut,fin = 73,73

# Boucle de simulation
while t < tmax :
    if etat == "etat_Consom":
        masse_grappe_moule = liste_masses_grappes_moules[0] 
        cadence_moule_par_heure = liste_cadences_moule_par_heure[0]
        quantite_moules_a_produire  = liste_quantites_moules_a_produire[0]
        
        # La cadence de consommation en kg/min du i-ème modèle
        cadence_consommation_kg_min = cadence_moule_par_heure / 60 * masse_grappe_moule  # en kg/min

        # Quantités de moules du i-ème modèle produits en une minute
        cadence_moule_par_minute = cadence_consommation_kg_min / masse_grappe_moule # en unités/min
        
        if  debut < t and  t < fin :
            print('t = ',t)
            print("Ici1", Mg, PFC_Consom, cadence_consommation_kg_min,quantite_moules_a_produire,quantite_moules_restants)
        
        # Si l'on peut consommer (c-a-d Mg et PFC sont okay) alors on consomme
        if (PFC_Consom - cadence_consommation_kg_min >= PFCmin) and (Mg - eC >= Mgmin):
            
            # Mise à jour du pourcentage de Mg et du poid fonte coulée
            PFC_Consom -= cadence_consommation_kg_min
            Mg -= eC

            # Quantités de moules restants du i-ème modèle après une minute
            quantite_moules_restants = quantite_moules_a_produire - cadence_moule_par_minute # en unités
            

            if  debut < t and  t < fin :
                print("Ici2", PFC_Consom, cadence_consommation_kg_min,quantite_moules_a_produire,quantite_moules_restants)

            
            # On a finis de réaliser le i-ème modèle alors on passe au i+1-ème modèle
            # Avec la mise à jour de liste_quantites_moules_a_produire et liste_masses_grappes_moules
            if quantite_moules_restants <= 0:
                # Passer à l'état changement de modèles
                etat = "etat_Serie"
                # Suppression des éléments déjà traités dans les listes
                liste_masses_grappes_moules.pop(0)
                liste_quantites_moules_a_produire.pop(0)
                liste_cadences_moule_par_heure.pop(0)
                
                # Si on a finis de tous produire alors on stoppe la procédure 
                # On passe  à l'état Fin
                if liste_masses_grappes_moules == []:  
                    etat = "etat_Fin" 
                
                if  debut < t and  t < fin :
                    print("Ici3",PFC_Consom, cadence_consommation_kg_min,quantite_moules_a_produire,quantite_moules_restants)

            # On a finis de réaliser le j-ème moules du i-ème modèle alors on passe au j-ème moules
            # En restant dans l'état consommation 
            else:
                if  debut < t and  t < fin :
                    print("Ici3",PFC_Consom, cadence_consommation_kg_min,quantite_moules_a_produire,quantite_moules_restants)

                liste_quantites_moules_a_produire[0] = quantite_moules_restants
                etat = "etat_Consom"
                
        else:
            # Ajout poche car Mg ou PFC insuffisant !!
            PFC_Consom += poche_i
            Mg = Mgmax 

            if numPoche < 4:
                numPoche += 1
            else :
                numPoche = 1
                numFours += 1
            
            # Mise à jour du pourcentage de Mg et du poid fonte coulée
            PFC_Consom -= cadence_consommation_kg_min
            Mg -= eC
            
            if PFC_Consom > PFCmax :
                print("On a depasser pas la capacité du Fours de Coulée")
            cadence_moule_par_minute = cadence_consommation_kg_min / masse_grappe_moule
            quantite_moules_restants = quantite_moules_a_produire - cadence_moule_par_minute
            
            if  debut < t and  t < fin :
                print("Ici2", Mg, PFC_Consom, cadence_consommation_kg_min,quantite_moules_a_produire,quantite_moules_restants)

            # On a finis de réaliser le i-ème modèle alors on passe au i+1-ème modèle
            # Avec la mise à jour de liste_quantites_moules_a_produire et liste_masses_grappes_moules
            if quantite_moules_restants <= 0:
                etat = "etat_Serie"
                # Suppression des éléments déjà traités dans les listes
                liste_masses_grappes_moules.pop(0)
                liste_quantites_moules_a_produire.pop(0)
                liste_cadences_moule_par_heure.pop(0)

                # Si on a finis de tous produire alors on stoppe la procédure 
                # On passe  à l'état Fin
                if liste_masses_grappes_moules == []:
                    etat = "etat_Fin"
                
                if  debut < t and  t < fin :
                    print("Ici3", Mg, PFC_Consom, cadence_consommation_kg_min,quantite_moules_a_produire,quantite_moules_restants)
            
            # On a finis de réaliser le j-ème moules du i-ème modèle alors on passe au j-ème moules
            # En restant dans l'état consommation 
            else:
                if  debut < t and  t < fin :
                    print("Ici3", Mg, PFC_Consom, cadence_consommation_kg_min,quantite_moules_a_produire,quantite_moules_restants)

                liste_quantites_moules_a_produire[0] = quantite_moules_restants
                etat = "etat_Consom"

    elif etat == "etat_Serie":
        # print("Changement de Serie")
        Mg -= eC
        temps_serie_courant -= 1
        
        if temps_serie_courant <= 0:
            etat = "etat_Consom"
            if  debut < t and  t < fin :
                print("Fin Temps Serie",Mg, PFC_Consom, cadence_consommation_kg_min,quantite_moules_restants)
            temps_serie_courant = temps_serie
        else:
            etat == "etat_Serie" 


    elif etat == "etat_Panne":
        Mg -= eC
        temps_Panne_courant -= 1
        
        if temps_Panne_courant <= 0:
            etat = "etat_Consom"
            if  debut < t and  t < fin :
                print("Fin Panne",Mg, PFC_Consom, cadence_consommation_kg_min,quantite_moules_restants)
            temps_Panne_courant = temps_Panne
        else:
            etat == "etat_Panne"

    elif etat == "etat_Fin":
        break
    
    # Mise à jour du temps
    t += 1
    cle = "Four 1"
    cle = f"Four {numFours}" 
    PFC_Consomvalues_dict[cle].append(PFC_Consom)
    Mgvalues_dict[cle].append(Mg)
    times_dict[cle].append(t)




# Affichage des résultats finaux ou mise à jour des dictionnaires
print("Résultats finaux ou mises à jour des dictionnaires : ")
print("times_dict : ", times_dict)
print("PFC_Consomvalues_dict : ", PFC_Consomvalues_dict)
print("Mgvalues_dict : ", Mgvalues_dict)


Résultats finaux ou mises à jour des dictionnaires : 
times_dict :  {'chargement_et_fusion': [], 'Four 1': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], 'Four 2': [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76], 'Four 3': [77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154], 'Four 4': [155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193,

In [14]:
import plotly.graph_objects as go

def Plot_consom(PFC_values_dict, Mgvalues_dict, times_dict, PFCmin, PFCmax, Mgmin, Mgmax):
    # Dictionnaire de couleurs pour chaque clé
    color_dict = {
        "chargement_et_fusion": 'royalblue',
        "Four 1": 'firebrick',
        "Four 2": 'green',
        "Four 3": 'orange',
        "Four 4": 'purple',
        "Four 5": 'royalblue'
    }

    # Create Plotly figure for PFC
    fig_pfc = go.Figure()

    # Add PFC lines for each key
    for key in PFC_values_dict.keys():
        fig_pfc.add_trace(go.Scatter(x=times_dict[key], y=PFC_values_dict[key], mode='lines', name=f'PFC over time ({key})', line=dict(color=color_dict[key])))

    # Add horizontal lines for PFCmin and PFCmax
    tmin, tmax = times_dict["Four 1"][0],times_dict["Four 1"][-1]
    fig_pfc.add_shape(type="line", x0=tmin, y0=PFCmin, x1=tmax, y1=PFCmin, line=dict(color="firebrick", width=2, dash="dash"), name="PFCmin")
    fig_pfc.add_shape(type="line", x0=tmin, y0=PFCmax, x1=tmax, y1=PFCmax, line=dict(color="green", width=2, dash="dash"), name="PFCmax")

    # Customize layout for PFC figure
    fig_pfc.update_layout(title='Evolution de la fonte dans le Four de Coulée au cours du temps (PFC)', 
                          xaxis_title='Time (minutes)', yaxis_title='PFC (kg)', 
                          template='plotly', showlegend=True,
                          annotations=[
                              dict(x=tmax, y=PFCmin, xref="x", yref="y", text="PFCmin", showarrow=True, arrowhead=2, ax=20, ay=-30),
                              dict(x=tmax, y=PFCmax, xref="x", yref="y", text="PFCmax", showarrow=True, arrowhead=2, ax=20, ay=-30)
                          ])

    # Show the PFC plot
    fig_pfc.show()

    # Create Plotly figure for Mg
    fig_mg = go.Figure()

    # Add Mg lines for each key
    for key in Mgvalues_dict.keys():
        fig_mg.add_trace(go.Scatter(x=times_dict[key], y=Mgvalues_dict[key], mode='lines', name=f'Mg over time ({key})', line=dict(color=color_dict[key])))

    # Add horizontal lines for Mgmin and Mgmax
    fig_mg.add_shape(type="line", x0=tmin, y0=Mgmin, x1=tmax, y1=Mgmin, line=dict(color="firebrick", width=2, dash="dash"), name="Mgmin")
    fig_mg.add_shape(type="line", x0=tmin, y0=Mgmax, x1=tmax, y1=Mgmax, line=dict(color="green", width=2, dash="dash"), name="Mgmax")

    # Customize layout for Mg figure
    fig_mg.update_layout(title='Evolution de la fonte dans le Four de Coulée au cours du temps (Mg)', 
                         xaxis_title='Time (minutes)', yaxis_title='Mg (kg)', 
                         template='plotly', showlegend=True,
                         annotations=[
                              dict(x=tmax, y=Mgmin, xref="x", yref="y", text="Mgmin", showarrow=True, arrowhead=2, ax=20, ay=-30),
                              dict(x=tmax, y=Mgmax, xref="x", yref="y", text="Mgmax", showarrow=True, arrowhead=2, ax=20, ay=-30)
                          ])

    # Show the Mg plot
    fig_mg.show()

# Call the function with the appropriate dictionaries and parameters
Plot_consom(PFC_Consomvalues_dict, Mgvalues_dict, times_dict, PFCmin, PFCmax, Mgmin, Mgmax)



initialiser t, tmax, Mg, Mgmin, Mgmax, eC, PFC_Consom, PFCmin, PFCmax, poche_i, debut, fin, etat, temps_serie, temps_Panne
initialiser liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire
initialiser PFC_Consomvalues_dict, Mgvalues_dict, times_dict, cle

tant que t < tmax faire
    si etat == "etat_Consom" alors
        définir masse_grappe_moule, cadence_moule_par_heure, quantite_moules_a_produire
        calculer cadence_consommation_kg_min
        calculer cadence_moule_par_minute

        si PFC_Consom - cadence_consommation_kg_min >= PFCmin et Mg - eC >= Mgmin alors
            mettre à jour PFC_Consom, Mg
            calculer quantite_moules_restants

            si quantite_moules_restants <= 0 alors
                changer état à "etat_Serie"
                supprimer les éléments déjà traités dans les listes

                si liste_masses_grappes_moules est vide alors
                    changer état à "etat_Fin"
                fin si

            sinon
                mettre à jour liste_quantites_moules_a_produire
            fin si
        sinon
            ajouter poche
            mettre à jour PFC_Consom, Mg

            calculer quantite_moules_restants

            si quantite_moules_restants <= 0 alors
                changer état à "etat_Serie"
                supprimer les éléments déjà traités dans les listes

                si liste_masses_grappes_moules est vide alors
                    changer état à "etat_Fin"
                fin si
            sinon
                mettre à jour liste_quantites_moules_a_produire
            fin si
        fin si

    sinon si etat == "etat_Serie" alors
        diminuer Mg, temps_serie_courant

        si temps_serie_courant <= 0 alors
            changer état à "etat_Consom"
            réinitialiser temps_serie_courant
        fin si

    sinon si etat == "etat_Panne" alors
        diminuer Mg, temps_Panne_courant

        si temps_Panne_courant <= 0 alors
            changer état à "etat_Consom"
            réinitialiser temps_Panne_courant
        fin si
    sinon si etat == "etat_Fin" alors
        arrêter la boucle
    fin si

    mettre à jour t, PFC_Consomvalues, Mgvalues, times
fin tant que


# Courbe interactive

In [3]:
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display
import asyncio
import time

def initialize_variables(initial_time_step, initial_t):
    """Initialise les variables de contrôle avec des paramètres."""
    global running, current_function, time_step, xdata, ydata, t
    running = False
    current_function = 0
    time_step = initial_time_step
    xdata = []
    ydata = []
    t = initial_t
def create_figure():
    """Crée et configure la figure Plotly."""
    fig = go.FigureWidget()
    scatter = fig.add_scatter(mode='lines+markers').data[0]
    fig.update_layout(title="Courbe Interactive", xaxis_title="Temps", yaxis_title="Valeur")
    return fig, scatter

async def update_data():
    """Fonction de mise à jour des données."""
    global running, current_function, time_step, xdata, ydata, t
    while True:
        if running:
            
            xdata.append(t)
            if current_function == 0:
                ydata.append(np.sin(t))
            elif current_function == 1:
                ydata.append(2)

            with fig.batch_update():
                scatter.update(x=xdata, y=ydata)
                
            t += time_step
            time.sleep(0.5)
        
        await asyncio.sleep(time_step)

def start_or_next_function(button):
    """Fonction pour démarrer ou changer la fonction."""
    global running, current_function
    current_function = (current_function + 1) % 2
    running = True

def stop_plot(button):
    """Fonction pour arrêter la mise à jour."""
    global running
    running = False

def reset_plot(button):
    """Fonction pour réinitialiser les données et le temps t."""
    global running, xdata, ydata, t
    running = False
    xdata = []
    ydata = []
    t = 0
    with fig.batch_update():
        scatter.update(x=xdata, y=ydata)

def create_buttons():
    """Crée les boutons interactifs."""
    start_button = widgets.Button(description="Start/Next Function")
    stop_button = widgets.Button(description="Stop")
    reset_button = widgets.Button(description="Reset")

    start_button.on_click(start_or_next_function)
    stop_button.on_click(stop_plot)
    reset_button.on_click(reset_plot)

    display(widgets.HBox([start_button, stop_button, reset_button]), fig)

def main(initial_time_step=1, initial_t=0):
    """Fonction principale pour démarrer l'application."""
    initialize_variables(initial_time_step, initial_t)
    global fig, scatter
    fig, scatter = create_figure()
    create_buttons()
    loop = asyncio.get_event_loop()
    loop.create_task(update_data())

# Appel de la fonction principale pour démarrer l'application
main()


FigureWidget({
    'data': [{'mode': 'lines+markers', 'type': 'scatter', 'uid': '4b20f932-f9cd-4e66-a9d9-861a420f72a0'}],
    'layout': {'template': '...',
               'title': {'text': 'Courbe Interactive'},
               'xaxis': {'title': {'text': 'Temps'}},
               'yaxis': {'title': {'text': 'Valeur'}}}
})

# Code 3

In [22]:
# Les Constantes du Programme
fonte_produit = 5000  
temps_chargement = 30  
temps_fusion = 45 
temps_traitement = 10

PFCmin = 2500 # en kg
PFCmax = 4000 # en kg
Mgmin = 0.035 # en %
Mgmax = 0.045 # en %
eC = 0.0005 # en %


liste_nbMoules_heure = [190, 190, 190, 210, 190, 190, 190, 190, 190] # en Tonnes/hr
liste_Quantite_Modeles = [121, 50, 35, 10, 100, 50, 35, 50, 40]
liste_Masse_Modeles = [41.2, 39.36, 15.6, 19.94, 41.20, 39.36, 15.60, 41.20, 29.65] # en kg


# Calcul de la liste des cadences
liste_cadences = [] # en Tonnes/hr
for nb_moules, masse_grappe in zip(liste_nbMoules_heure, liste_Masse_Modeles):
    cadence = nb_moules / 60 * masse_grappe
    liste_cadences.append(cadence)

    
poche_i = 1250 # en kg


In [23]:
import plotly.graph_objects as go
import numpy as np

def Plot_consom(PFC_values_dict, times_dict, PFCmin, PFCmax):
    # Dictionnaire de couleurs pour chaque clé
    color_dict = {
        "chargement_et_fusion": 'royalblue',
        "Four 1": 'firebrick',
        "Four 2": 'green',
        "Four 3": 'orange',
        "Four 4": 'purple'
    }

    # Create Plotly figure
    fig = go.Figure()

    # Add PFC lines for each key
    for key in PFC_values_dict.keys():
        fig.add_trace(go.Scatter(x=times_dict[key], y=PFC_values_dict[key], mode='lines', name=f'PFC over time ({key})', line=dict(color=color_dict[key])))

    # Add horizontal lines for PFCmin and PFCmax
    tmin, tmax = times_dict["chargement_et_fusion"][0],times_dict["Four 1"][-1]
    fig.add_shape(type="line", x0=tmin, y0=PFCmin, x1=tmax, y1=PFCmin, line=dict(color="firebrick", width=2, dash="dash"), name="PFCmin")
    fig.add_shape(type="line", x0=tmin, y0=PFCmax, x1=tmax, y1=PFCmax, line=dict(color="green", width=2, dash="dash"), name="PFCmax")

    # Customize layout
    fig.update_layout(title='Evolution de la fonte dans le Four de Coulée au cours du temps', 
                      xaxis_title='Time (minutes)', yaxis_title='PFC (kg)', 
                      template='plotly', showlegend=True,
                      annotations=[
                          dict(x=tmax, y=PFCmin, xref="x", yref="y", text="PFCmin", showarrow=True, arrowhead=2, ax=20, ay=-30),
                          dict(x=tmax, y=PFCmax, xref="x", yref="y", text="PFCmax", showarrow=True, arrowhead=2, ax=20, ay=-30)
                      ])

    # Show the plot
    fig.show()

def Plot_Prod(PFC_values_dict, times_dict):
    # Dictionnaire de couleurs pour chaque clé
    color_dict = {
        "chargement_et_fusion": 'royalblue',
        "Four 1": 'firebrick',
        "Four 2": 'green',
        "Four 3": 'orange',
        "Four 4": 'purple'
    }

    # Create Plotly figure
    fig = go.Figure()

    # Add PFC lines for each key
    for key in PFC_values_dict.keys():
        fig.add_trace(go.Scatter(x=times_dict[key], y=PFC_values_dict[key], mode='lines', name=f'PFC over time ({key})', line=dict(color=color_dict[key])))

    # Customize layout
    fig.update_layout(title='Evolution de la fonte disponible dans le Four de Fusion au cours du temps', 
                      xaxis_title='Time (minutes)', yaxis_title='PFC (kg)', 
                      template='plotly', showlegend=True)

    # Show the plot
    fig.show()


In [24]:


def update_production_chargement_et_fusion(t, numfours, PFC_Prod):
    
    """
    Met à jour la production de fonte en fonction du temps écoulé lors du chargement et de la fusion.
    
    :param t: Temps actuel en minutes
    :param Temps_chargement: Temps de chargement en minutes
    :param Temps_fusion: Temps de fusion en minutes
    :param numfours: Nombre de fours en production
    :param Fonte_produit: Quantité de fonte produite par cycle en kg
    :param PFC_Prod: Production actuelle de fonte en kg
    :param poche: Paramètre optionnel, par défaut à 0
    :return: Tuple (numfours, PFC_Prod) mis à jour
    """
    # if t >= Temps_chargement + Temps_fusion:
    if t >= temps_chargement + temps_fusion:
        numfours += 1
        PFC_Prod += fonte_produit
    return numfours, PFC_Prod

def calcule_chargement_et_fusion(times_dict, PFC_Prodvalues_dict, PFC_Consomvalues_dict, numfours):
    """
    Calcule la consommation et la production de fonte du début du chargement à la fin de la 1ère fusion.
    """
    # Derniers éléments des listes pour la clé 'chargement_et_fusion'
    PFC_Consom = PFC_Consomvalues_dict['chargement_et_fusion'][-1] if times_dict['chargement_et_fusion'] else 0
    PFC_Prod = PFC_Prodvalues_dict['chargement_et_fusion'][-1] if times_dict['chargement_et_fusion'] else 0
    t = times_dict['chargement_et_fusion'][-1] if times_dict['chargement_et_fusion'] else 0

    # Mise à jour de la production de fonte après chargement
    t += temps_chargement
    numfours, PFC_Prod = update_production_chargement_et_fusion(t, numfours, PFC_Prod)
    PFC_Prodvalues_dict['chargement_et_fusion'].append(PFC_Prod)
    PFC_Consomvalues_dict['chargement_et_fusion'].append(PFC_Consom)
    times_dict['chargement_et_fusion'].append(t)

    # # Mise à jour de la production de fonte entre début/fin fusion
    # t += temps_fusion - 1
    # numfours, PFC_Prod = update_production_chargement_et_fusion(t, numfours, PFC_Prod)
    # PFC_Prodvalues_dict['chargement_et_fusion'].append(PFC_Prod)
    # PFC_Consomvalues_dict['chargement_et_fusion'].append(PFC_Consom)
    # times_dict['chargement_et_fusion'].append(t)

    # # Mise à jour de la production de fonte après fusion
    # t += 1
    # numfours, PFC_Prod = update_production_chargement_et_fusion(t, numfours, PFC_Prod)
    # PFC_Prodvalues_dict['chargement_et_fusion'].append(PFC_Prod)
    # PFC_Consomvalues_dict['chargement_et_fusion'].append(PFC_Consom)
    # times_dict['chargement_et_fusion'].append(t)

    # Mise à jour de la production de fonte après fusion
    t += temps_fusion
    numfours, PFC_Prod = update_production_chargement_et_fusion(t, numfours, PFC_Prod)
    PFC_Prodvalues_dict['chargement_et_fusion'].append(PFC_Prod)
    PFC_Consomvalues_dict['chargement_et_fusion'].append(PFC_Consom)
    times_dict['chargement_et_fusion'].append(t)

    return times_dict, PFC_Prodvalues_dict, PFC_Consomvalues_dict

def update_production(t, numfours, PFC_Prod, poche = 0):
    """
    Met à jour la production de fonte en fonction du temps écoulé.
    """
    if t > temps_chargement and (t - temps_chargement) % temps_fusion == 0 and numfours <= 4 :
        numfours += 1
        PFC_Prod += fonte_produit
    PFC_Prod -= poche
    return numfours, PFC_Prod

def calcule_Transitoire(times_dict, PFC_Prodvalues_dict, PFC_Consomvalues_dict, numfours):
    """
    Calcule la consommation et la production de fonte lors des 3 premières poches (Régime Transitoire).
    """
    # Temps de traitement et poches
    temps_poche_1, temps_poche_2, temps_poche_3 = 10, 10, 10
    poche_1, poche_2, poche_3 = 1250, 1250, 1250

    # Déterminer la clé basée sur numfours
    cle = f"Four {numfours}" if numfours in [1, 2, 3, 4] else "chargement_et_fusion"

    # Derniers éléments des listes pour la clé correspondante
    PFC_Consom = PFC_Consomvalues_dict["chargement_et_fusion"][-1] 
    PFC_Prod = PFC_Prodvalues_dict["chargement_et_fusion"][-1] 
    t = times_dict["chargement_et_fusion"][-1] 


    # Déterminer la clé basée sur numfours
    cle = f"Four {numfours}" if numfours in [1, 2, 3, 4] else "chargement_et_fusion"
    times_dict[cle].append(t)
    PFC_Prodvalues_dict[cle].append(PFC_Prod)
    PFC_Consomvalues_dict[cle].append(PFC_Consom)

    # Poche 1
    t += temps_poche_1
    PFC_Consom += poche_1
    numfours, PFC_Prod = update_production(t, numfours, PFC_Prod, poche_1)
    PFC_Prodvalues_dict[cle].append(PFC_Prod)
    PFC_Consomvalues_dict[cle].append(PFC_Consom)
    times_dict[cle].append(t)

    # Poche 2
    t += temps_poche_2
    PFC_Consom += poche_2
    PFC_Consomvalues_dict[cle].append(PFC_Consom)
    numfours, PFC_Prod = update_production(t, numfours, PFC_Prod, poche_2)
    PFC_Prodvalues_dict[cle].append(PFC_Prod)
    times_dict[cle].append(t)

    # Poche 3
    t += temps_poche_3
    PFC_Consom += poche_3
    PFC_Consomvalues_dict[cle].append(PFC_Consom)
    numfours, PFC_Prod = update_production(t, numfours, PFC_Prod, poche_3)
    PFC_Prodvalues_dict[cle].append(PFC_Prod)
    times_dict[cle].append(t)

    return times_dict, PFC_Prodvalues_dict, PFC_Consomvalues_dict






def calcule_temps_poche_i(PFC_Consomvalues_dict, Mg, numfours=1):
    """
    Calcule le temps nécessaire pour ajouter la poche i.

    Args:
        PFC_Consomvalues_dict (dict): Dictionnaire contenant les valeurs de consommation de PFC par clé.
        Mg (float): Valeur actuelle de Mg.
        Mgmin (float): Valeur minimale acceptable de Mg.
        PFCmin (float): Valeur minimale acceptable de PFC.
        eC (float): Variation de Mg par unité de temps.
        liste_Masse_Modeles (list of float): Liste des masses des grappes.
        liste_Quantite_Modeles (list of int): Liste des quantités de mottes pour chaque grappe.
        liste_cadences (list of float): Liste des cadences pour chaque grappe.
        numfours (int): Numéro du four à utiliser (1, 2, 3, ou 4). Par défaut à 1.

    Returns:
        int: Temps nécessaire pour ajouter la poche 4.
    """
    cle = f"Four {numfours}"
    if cle not in PFC_Consomvalues_dict or not PFC_Consomvalues_dict[cle]:
        raise ValueError(f"Clé '{cle}' invalide ou valeurs de consommation manquantes")


    temps_serie = 3
    ObjetjModeles_i = 1


    PFC_Consom = PFC_Consomvalues_dict[cle][-1]
    temps_ajout_poche_i_Coulee = 0
    for idx, (masse_grappe, quantite_mottes) in enumerate(zip(liste_Masse_Modeles, liste_Quantite_Modeles)):
        cadence = liste_cadences[idx]
        for _ in range(quantite_mottes):
            PFC_Consom -= cadence
            Mg -= eC
            if PFC_Consom <= PFCmin or Mg <= Mgmin:
                return temps_ajout_poche_i_Coulee
            temps_ajout_poche_i_Coulee += 1
        temps_ajout_poche_i_Coulee += temps_serie
    return temps_ajout_poche_i_Coulee


def calcule_temps_poche_i(PFC_Consom, Mg, numfours=1):
    """
    Calcule le temps nécessaire pour ajouter la poche i.

    Args:
        PFC_Consom (float): Valeur de consommation de PFC.
        Mg (float): Valeur actuelle de Mg.
        numfours (int): Numéro du four à utiliser (1, 2, 3, ou 4). Par défaut à 1.

    Returns:
        list: Liste des temps nécessaires pour chaque changement de série.
    """

    Pause_serie = 30
    temps_ajout_poche_i_Coulee = []
    stop = False
    for idx, (masse_grappe, quantite_mottes) in enumerate(zip(liste_Masse_Modeles, liste_Quantite_Modeles)):
        cadence = liste_cadences[idx]
        temps_serie_courant = 0
        for _ in range(quantite_mottes):
            PFC_Consom -= cadence
            Mg -= eC
            if  Mg <= Mgmin: # stop à cause de Mgmin
                print("Mg")

            if PFC_Consom <= PFCmin or Mg <= Mgmin:
                temps_ajout_poche_i_Coulee.append(("temps_serie_courant", temps_serie_courant))
                stop  = True
                break
            temps_serie_courant += 1
        if stop :
            break
        Mg -= Pause_serie*eC
        if len(liste_Quantite_Modeles) >= 2: # Si c'est le dernier element plus besoin d'une Pause_serie
            temps_ajout_poche_i_Coulee.append(("Pause_serie", Pause_serie))  
    # print(temps_ajout_poche_i_Coulee)
    return temps_ajout_poche_i_Coulee



def Consomation_fonteCoulee(PFC_Consom, Mg, ObjetjModeles_i):
    """
    Met à jour la consommation de fonte, Mg, et les listes des modèles en fonction des conditions spécifiées.

    Args:
        PFC_Consom (float): Consommation actuelle de fonte.
        Mg (float): Quantité actuelle de Mg.
        eC (float): Variation de Mg par unité de temps.
        ObjetjModeles_i (int): Index du modèle actuel.
        liste_Masse_Modeles (list of float): Liste des masses des grappes.
        liste_Quantite_Modeles (list of int): Liste des quantités de mottes pour chaque grappe.
        liste_cadences (list of float): Liste des cadences pour chaque grappe.

    Returns:
        tuple: Mise à jour de (PFC_Consom, Mg, ObjetjModeles_i, liste_Masse_Modeles, liste_Quantite_Modeles).
    """
    FinConso = False

    # Panne = False

    if liste_Masse_Modeles != []:
        masse_grappe = liste_Masse_Modeles[0]
        cadence = liste_cadences[0]
        # print("Serie ", masse_grappe, ObjetjModeles_i)

        # On est sur la Conception du Modeles i
        if ObjetjModeles_i > 1:
            # Mise à jour de la consommation de fonte et de Mg
            PFC_Consom -= cadence
            Mg -= eC
            ObjetjModeles_i -= 1
        # On est sur la Conception du Modeles i+1
        # Donc pris en compte du temps_serie
        else:
            # Gestion du Changement de Serie
            print("Gestion du Changement de Serie")
            # Suppression des éléments déjà traités dans les listes
            liste_Masse_Modeles.pop(0)
            liste_Quantite_Modeles.pop(0)
            liste_cadences.pop(0)

            # Mise à jour des valeurs
            if liste_Quantite_Modeles:
                ObjetjModeles_i = liste_Quantite_Modeles[0]

    else :
        FinConso = True
        # On stop tous le processus

    return PFC_Consom, Mg, ObjetjModeles_i,FinConso


def gestion_poche_et_consomation(numfours, PFC_Prodvalues_dict, PFC_Consomvalues_dict, times_dict,liste_Quantite_Modeles):
    """
    Gère l'ajout de poche, la consommation de fonte et la mise à jour des temps et valeurs correspondantes.

    Args:

    Returns:
        tuple: Mise à jour de .
    """
    
    cle = f"Four {numfours}" if numfours in [1, 2, 3, 4] else "chargement_et_fusion"

    # Derniers éléments des listes pour la clé correspondante
    PFC_Consom = PFC_Consomvalues_dict[cle][-1] 
    PFC_Prod = PFC_Prodvalues_dict[cle][-1] 
    t = times_dict[cle][-1] 

    # Ajout poche_i
    PFC_Consom += poche_i
    Mg = Mgmax

    # Consommation de fonte lors du transvasement poche coulée vers Fours coulée
    ObjetjModeles_i = liste_Quantite_Modeles[0]
    PFC_Consom, Mg, ObjetjModeles_i,FinConso =  Consomation_fonteCoulee(PFC_Consom, Mg, ObjetjModeles_i)
    liste_Quantite_Modeles[0] = ObjetjModeles_i

    # Quand ajout poche_i+1 ?
    temps_ajout_poche_i_Coulee = calcule_temps_poche_i(PFC_Consom, Mg, numfours=1)


    # Mise à jour du temps
    t += 1
    PFC_Prodvalues_dict[cle].append(PFC_Prod)
    PFC_Consomvalues_dict[cle].append(PFC_Consom)
    times_dict[cle].append(t)

    return times_dict,PFC_Consomvalues_dict, PFC_Prodvalues_dict, Mg, temps_ajout_poche_i_Coulee,FinConso


def gestion_consomation_fonte_coulee(PFC_Consomvalues_dict, PFC_Prodvalues_dict, times_dict, numfours, temps_ajout_poche_i_Coulee, Mg):
    """
    Gère la consommation de fonte dans les fours de coulée entre 2 poches.

    Args:


    Returns:
        tuple: 
    """
    cle = f"Four {numfours}" if numfours in [1, 2, 3, 4] else "chargement_et_fusion"
    
    # numpoche = 0
    # if numpoche  == 4 :
    #     numfours+=1
    #     numpoche = 1

    #     cle = f"Four {numfours}" if numfours in [1, 2, 3, 4] else "chargement_et_fusion"
    #     PFC_Prodvalues_dict[cle].append(PFC_Prod)
    #     PFC_Consomvalues_dict[cle].append(PFC_Consom)
    #     times_dict[cle].append(t)

    # else :
    #     numpoche += 1

    # Derniers éléments des listes pour la clé correspondante
    PFC_Consom = PFC_Consomvalues_dict[cle][-1] 
    PFC_Prod = PFC_Prodvalues_dict[cle][-1] 
    t = times_dict[cle][-1] 


    FinConso = False


    for i, t_iter in enumerate(temps_ajout_poche_i_Coulee):
        if t_iter[0] == 'temps_serie_courant':
            ObjetjModeles_i = liste_Quantite_Modeles[0]
            cadence = liste_cadences[0]
            if t_iter[1] == 0 : 
                # Si temps_ajout_poche_i_Coulee = 0 (cela veut dire que l'on doit
                # consomme immediatement )
                # Alors consommer jusqu'à la valeur min
                while PFC_Consom-cadence >= PFCmin or Mg-eC >= Mgmin : 
                    ### Consommation de fonte !!
                    PFC_Consom, Mg, ObjetjModeles_i,FinConso = Consomation_fonteCoulee(PFC_Consom, Mg, ObjetjModeles_i)

                    # Mise à jour du temps
                    t += 1
                    PFC_Prodvalues_dict[cle].append(PFC_Prod)
                    PFC_Consomvalues_dict[cle].append(PFC_Consom)
                    times_dict[cle].append(t)

                liste_Quantite_Modeles[0] = ObjetjModeles_i
            else :
                # Sinon consomme jusqu'a la durrée avant l'ajout de poche
                t_fin = t + t_iter[1]
                ### Consommation de fonte dans le Fours de Coulée!!
                while t < t_fin:
                    ### Consommation de fonte !!
                    PFC_Consom, Mg, ObjetjModeles_i,FinConso = Consomation_fonteCoulee(PFC_Consom, Mg, ObjetjModeles_i)

                    # Mise à jour du temps
                    t += 1
                    PFC_Prodvalues_dict[cle].append(PFC_Prod)
                    PFC_Consomvalues_dict[cle].append(PFC_Consom)
                    times_dict[cle].append(t)

                liste_Quantite_Modeles[0] = ObjetjModeles_i

        if t_iter[0] == 'Pause_serie':  
            Mg -= t_iter[1]*eC

            t += t_iter[1]      
            PFC_Prodvalues_dict[cle].append(PFC_Prod)
            PFC_Consomvalues_dict[cle].append(PFC_Consom)
            times_dict[cle].append(t)

    return PFC_Consomvalues_dict, PFC_Prodvalues_dict, times_dict, Mg,FinConso



In [28]:

# Initialisation des dictionnaires
cles = ["chargement_et_fusion", "Four 1", "Four 2", "Four 3", "Four 4"]
times_dict = {cle: [] for cle in cles}
PFC_Consomvalues_dict = {cle: [] for cle in cles}
PFC_Prodvalues_dict = {cle: [] for cle in cles}
times_dict["chargement_et_fusion"] = [0]
PFC_Consomvalues_dict["chargement_et_fusion"] = [0]
PFC_Prodvalues_dict["chargement_et_fusion"] = [0]



numfours = 1  # Exemple de numéro de four (peut être 1, 2, 3 ou 4)
poche_i = 1250


liste_nbMoules_heure = [190, 190, 190, 210, 190, 190, 190, 190, 190] # en Tonnes/hr
liste_Quantite_Modeles = [121, 50, 35, 10, 100, 50, 35, 50, 40]
liste_Masse_Modeles = [41.2, 39.36, 15.6, 19.94, 41.20, 39.36, 15.60, 41.20, 29.65] # en kg


# Calcul de la liste des cadences
liste_cadences = [] # en kg/min
for nb_moules, masse_grappe in zip(liste_nbMoules_heure, liste_Masse_Modeles):
    cadence = nb_moules / 60 * masse_grappe
    liste_cadences.append(cadence)

In [29]:
times_dict, PFC_Prodvalues_dict, PFC_Consomvalues_dict = calcule_chargement_et_fusion(times_dict, PFC_Prodvalues_dict, PFC_Consomvalues_dict, numfours)

times_dict, PFC_Prodvalues_dict, PFC_Consomvalues_dict = calcule_Transitoire(times_dict, PFC_Prodvalues_dict, PFC_Consomvalues_dict, numfours)


Mg = Mgmax
cle = f"Four {numfours}" if numfours in [1, 2, 3, 4] else "chargement_et_fusion"
PFC_Consom = PFC_Consomvalues_dict[cle][-1] 
# temps_ajout_poche_i_Coulee = calcule_temps_poche_i(PFC_Consomvalues_dict, Mg, liste_Masse_Modeles, liste_Quantite_Modeles, liste_cadences, numfours=1)
temps_ajout_poche_i_Coulee = calcule_temps_poche_i(PFC_Consom, Mg, numfours=1)


PFC_Consomvalues_dict, PFC_Prodvalues_dict, times_dict, Mg,FinConso  = gestion_consomation_fonte_coulee(PFC_Consomvalues_dict, PFC_Prodvalues_dict, times_dict, numfours, temps_ajout_poche_i_Coulee, Mg)

#### Temps/Moment où la poche_i est entierement ajoutée (et on consomme aussi) 
times_dict,PFC_Consomvalues_dict, PFC_Prodvalues_dict, Mg, temps_ajout_poche_i_Coulee,FinConso  = gestion_poche_et_consomation(numfours, PFC_Prodvalues_dict, PFC_Consomvalues_dict, times_dict,liste_Quantite_Modeles)

FinConso = False

nbFours = 3
for i in range(nbFours*16) : 
    # ### Consomation de fonte dans le Fours de Coulée!!
    PFC_Consomvalues_dict, PFC_Prodvalues_dict, times_dict, Mg,FinConso  = gestion_consomation_fonte_coulee(PFC_Consomvalues_dict, PFC_Prodvalues_dict, times_dict, numfours, temps_ajout_poche_i_Coulee, Mg)

    #### Temps/Moment où la poche_i est entierement ajoutée (et on consomme aussi) 
    times_dict,PFC_Consomvalues_dict, PFC_Prodvalues_dict, Mg, temps_ajout_poche_i_Coulee,FinConso  = gestion_poche_et_consomation(numfours, PFC_Prodvalues_dict, PFC_Consomvalues_dict, times_dict,liste_Quantite_Modeles)
    # numfours+=1
    # print(numfours)
    if FinConso :
        break


Plot_consom(PFC_Consomvalues_dict, times_dict, PFCmin, PFCmax)
Plot_Prod(PFC_Prodvalues_dict, times_dict)

Mg
Gestion du Changement de Serie
Mg
Gestion du Changement de Serie
Mg
Mg
Gestion du Changement de Serie
Gestion du Changement de Serie
Mg
Gestion du Changement de Serie
Mg
Gestion du Changement de Serie
Mg
Mg
Gestion du Changement de Serie
Gestion du Changement de Serie


In [41]:


def ProdConsom(liste_masse_grappe, liste_quantite_mottes, nb_moules_heure, t_serie, temps_fusion, temps_chargement, temps_traitement, Mg, Mgmin,Mgmax, eC):
    """
    Calcule la consommation et la production de fonte.
    """
    # Initialisation des variables
    numfours = 1
    fonte_produit = 5000  # Fonte produite toutes les 45 min en kg
    PFC_Prodvalues = [0]  # Production de fonte en kg
    PFC_Consomvalues = [0]  # Consommation de fonte en kg
    times = [0]  # Temps en minutes

    Listetemps_ajout_poche_i_Coulee = []
    Listetemps_depart_poche_i_Fusion = []

    # Initialiser les clés
    cles = ["chargement_et_fusion","Four 1", "Four 2", "Four 3", "Four 4"]
    times_dict = {}
    PFC_Consomvalues_dict = {}
    PFC_Prodvalues_dict = {}
    # PFC_Consomvalues = {cle: [0] for cle in cles}

    times, PFC_Prodvalues,PFC_Consomvalues = calcule_chargement_et_fusion(times, PFC_Prodvalues, PFC_Consomvalues, fonte_produit, numfours, temps_chargement, temps_fusion)


    times, PFC_Prodvalues,PFC_Consomvalues = calcule_Transitoire(times, PFC_Prodvalues, PFC_Consomvalues, fonte_produit, numfours, temps_chargement, temps_fusion)

    temps_ajout_poche_4_Coulee = calcule_temps_poche_4(PFC_Consomvalues, liste_masse_grappe, liste_quantite_mottes, Mgmax, Mgmin, PFCmin, eC)
    temps_depart_poche_i_Fusion = temps_ajout_poche_4_Coulee - temps_traitement
    Listetemps_ajout_poche_i_Coulee.append(temps_ajout_poche_4_Coulee)
    Listetemps_depart_poche_i_Fusion.append(temps_depart_poche_i_Fusion)
    
    numfours += 1




    PFC_Prod = PFC_Prodvalues[-1]
    PFC_Consom = PFC_Consomvalues[-1]
    t = times[-1]

    t_avantpoche_i = t
    poche_i = 1250

    # print( "ICI",PFC_Consom,t,Listetemps_ajout_poche_i_Coulee)


# ----------------------------------------------------------------------------------

    # Boucle principale consomation et production
    # Indices de suivi pour les listes
    t_max = 600 #4*60
    index_grappe = 1 #0
    index_motte = 1 #0
    quantite_mottes = liste_quantite_mottes[index_grappe-1]

    liste_nbMoules_heure = [190, 190, 190, 210, 190, 190, 190, 190, 190] # en Tonnes/hr
    liste_masse_grappe = [41.2, 39.36, 15.6, 19.94, 41.20, 39.36, 15.60, 41.20, 29.65] # en kg

    # Calcul de la liste des cadences
    liste_cadences = [] # en Tonnes/hr
    for nb_moules, masse_grappe in zip(liste_nbMoules_heure, liste_masse_grappe):
        cadence = nb_moules / 60 * masse_grappe
        liste_cadences.append(cadence)


    while t < t_max:  
        if index_grappe <= len(liste_masse_grappe) and index_motte <= quantite_mottes:
            masse_grappe = liste_masse_grappe[index_grappe-1]
            cadence =  liste_cadences[index_grappe-1]

            

            # print("Changement de serie ",index_grappe,index_motte)
            # print(PFC_Consom,t,Listetemps_ajout_poche_i_Coulee)

            # print( "ICI1",PFC_Consom,cadence,t,t_avantpoche_i,Listetemps_ajout_poche_i_Coulee)

            # Avant de consommer de la fonte, on doit savoir :
            # Quand ajouter la poche_i dans le Fours de Coulée ?
            if t == (Listetemps_ajout_poche_i_Coulee[-1] + t_avantpoche_i):
                PFC_Consom += poche_i
                Mg = Mgmax

                # print( "ICI2",PFC_Consom,cadence,t,t_avantpoche_i,Listetemps_ajout_poche_i_Coulee)

                # Calcul Temps prochain poche_i
                temps_ajout_poche_i_Coulee = calcule_temps_poche_i(PFC_Consom, index_grappe, liste_masse_grappe, index_motte, liste_quantite_mottes, Mg, Mgmin, PFCmin, eC)
                Listetemps_ajout_poche_i_Coulee.append(temps_ajout_poche_i_Coulee)
                t_avantpoche_i = t


            # Mise à jour des indices de suivi des mottes
            if index_motte  < quantite_mottes:
                index_motte += 1
            else:
                # Gestion du Changement de Serie
                # print("Changement de serie 2")

                # print("la", index_grappe)

                index_motte = 1
                index_grappe += 1
                if index_grappe <= len(liste_quantite_mottes):
                    quantite_mottes = liste_quantite_mottes[index_grappe-1]



            # Mise à jour de la consommation de fonte et de Mg
            PFC_Consom -= cadence  
            Mg -= eC



        # Mise à jour du temps
        t += 1
        # print("t = ",t)
        times.append(t)
        PFC_Prodvalues.append(PFC_Prod)
        PFC_Consomvalues.append(PFC_Consom)



    tmin = times[0]
    tmax = times[-1]
    return times, PFC_Consomvalues, PFC_Prodvalues, tmin, tmax


In [42]:
nb_fours = 6 # 8-9 fours pour HWS, 6 fours pour DiSA
temps_fusion = 45 # en min
temps_chargement = 30 # 30 - 45  min

# Les valeurs d'entrée Input 

nb_moules_heure = 190 
t_serie = 5 # 5 min

PFCmin = 2500 # en kg
PFCmax = 4000 # en kg

Mgmin = 0.035 # en %
Mgmax = 0.045 # en %
eC = 0.0005 # en %
temps_traitement = 10  # en  min



Mg = Mgmax

liste_masse_grappe = [41.2, 39.36, 15.6, 19.94, 41.20, 39.36, 15.60, 41.20, 29.65] # en kg
liste_quantite_mottes = [121, 50, 35, 10, 100, 50, 35, 50, 40]

liste_cadences = [7.83, 7.48, 2.96, 4.19, 7.83, 7.48, 2.96, 7.83, 5.63] # en Tonnes/hr

liste_temps_changement_serie = [3, 3, 3, 3, 3, 3, 3, 3] # en minutes


# liste_masse_grappe = [41.2, 39.36, 15.6] # en kg
# liste_quantite_mottes = [1*121, 50, 35]


# liste_masse_grappe = [35, 35] # en kg
# liste_quantite_mottes = [50, 50]

# liste_masse_grappe = [35] # en kg
# liste_quantite_mottes = [200]

times, PFC_Consomvalues, PFC_Prodvalues, tmin, tmax = ProdConsom(liste_masse_grappe, liste_quantite_mottes, nb_moules_heure, t_serie, temps_fusion, temps_chargement, temps_traitement, Mg, Mgmin,Mgmax, eC)

In [43]:
# Plot_Prod(times, PFC_Prodvalues)
Plot_consom(times, PFC_Consomvalues, tmin, tmax, PFCmin, PFCmax)

# Autres Brouillon

In [ ]:

Tu es un mathématicien et du dois effectuer cette tache suivant :

Faire varier la fonction Consomation_fonteCoulee au cours du temps en min
En conservant le temps t, La consomation PFC_Consom et 
le pourcentage de Mg dans un dictionnaire : times_dict, PFC_Consomvalues_dict, Mgvalues_dict
On souhaite interompre la consomation avec une variable la Panne :temps_Panne.
Pendant la Panne : le pourcentage de Mg continue à varier.
Cependant la panne peut intervenir à n'importe quel momment dans le temps et peut etre recurent, 
c'est un bouton que l'utilisateurs active ou non en donnant la durrée de la Panne temps_Panne


n'hésite pas a modifier  Consomation_fonteCoulee pour repondre à ses besoins !!!
Si tu as des questions ou s'il te manque certaines informations, n'hésite pas à me les demander !!


def Consomation_fonteCoulee(PFC_Consom, Mg, ObjetjModeles_i):
    """
    Met à jour la consommation de fonte, Mg, et les listes des modèles en fonction des conditions spécifiées.

    Args:
        PFC_Consom (float): Consommation actuelle de fonte.
        Mg (float): Quantité actuelle de Mg.
        eC (float): Variation de Mg par unité de temps.
        ObjetjModeles_i (int): Index du modèle actuel.
        liste_Masse_Modeles (list of float): Liste des masses des grappes.
        liste_Quantite_Modeles (list of int): Liste des quantités de mottes pour chaque grappe.
        liste_cadences (list of float): Liste des cadences pour chaque grappe.

    Returns:
        tuple: Mise à jour de (PFC_Consom, Mg, ObjetjModeles_i, liste_Masse_Modeles, liste_Quantite_Modeles).
    """

    if liste_Masse_Modeles != []:
        masse_grappe = liste_Masse_Modeles[0]
        cadence = liste_cadences[0]
        # print("Serie ", masse_grappe, ObjetjModeles_i)

        # On est sur la Conception du Modeles i
        if ObjetjModeles_i > 1:
            # Mise à jour de la consommation de fonte et de Mg
            PFC_Consom -= cadence
            Mg -= eC
            ObjetjModeles_i -= 1
        # On est sur la Conception du Modeles i+1
        # Donc pris en compte du temps_serie
        else:
            # Gestion du Changement de Serie
            print("Gestion du Changement de Serie")
            # Suppression des éléments déjà traités dans les listes
            liste_Masse_Modeles.pop(0)
            liste_Quantite_Modeles.pop(0)
            liste_cadences.pop(0)

            # Mise à jour des valeurs
            if liste_Quantite_Modeles:
                ObjetjModeles_i = liste_Quantite_Modeles[0]


    return PFC_Consom, Mg, ObjetjModeles_i


Merci de m'avoir bien repondu !! 
On souhaite ajouter maintenant ajouter 2 autre variables PFC_Prod et temps_ajout_poche_i_Coulee
Avec deux constantes poche_i = 1250, fonte_produit = 5000 
Toute les 70 minutes , on ajoutera fonte_produit à PFC_Prod 
et Toute les temps_ajout_poche_i_Coulee minutes on lui enlèvera poche_i
On souhaite conserver PFC_Prod dans un dictionnaire PFC_Consomvalues_dict 
et temps_ajout_poche_i_Coulee dans une liste ou autre. 
Si tu as des questions ou s'il te manque certaines informations, n'hésite pas à me les demander !!

